# Проект, Часть 1
## Царькова Анастасия, Скоробогатов Денис, Анвардинов Шариф

Для выполнения этого кода потребуются nltk (токенизация и стоп-слова), requests (http) и readability (выделение основного контента).

In [1]:
import re
import html
import json
import collections
import multiprocessing

from datetime import datetime, timedelta
from urllib.parse import urlparse

import requests
import readability

import nltk
import nltk.corpus

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/pyos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/pyos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### 1. HackerNews

**Задача**: выгрузить с HackerNews статьи со ссылками, комментарии, число лайков, даты публикации; перейти по каждой ссылке и сохранить основной контент статьи. Затем отфильтровать статьи, относящиеся к какому-то бренду.

In [3]:
HN_API_URL = 'https://hacker-news.firebaseio.com/v0/item/{}.json'
HN_API_MAX = 'https://hacker-news.firebaseio.com/v0/maxitem.json'

def traverse_comments(session, cur):
    '''
        Fetch & flatten the comment tree, starting with a given node.
    
        :param session: a requests session.
        :param cur: the tree root.

    '''
    for id in cur.get('kids', []):
        res = session.get(HN_API_URL.format(id)).json()
        if res is None:
            continue
        if not res.get('deleted'):
            yield res['text']
        for text in traverse_comments(session, res):
            yield text

def get_text(session, url):
    '''
        Fetch & extract main content from an external source.
        
        :param session: a requests session.
        :param url: the url of an article.

    '''
    try:
        return readability.Document(session.get(url).text).summary()
    except Exception:
        return 'error: could not fetch {}'.format(url)

Для ускорения краулить будем в 10 потоков. Разумеется, было бы оптимальней использовать asyncio, но это бы слишком сильно замедлило процесс разработки. Поскольку API у HackerNews оказался очень уж медленным, а времени довольно мало, сохранять статьи будем за последнюю неделю, а не за полгода.

In [4]:
def get_articles(i, w, n, output):
    '''
        Fetch HackerNews articles with comments.

        :param i: the id of the worker thread.
        :param w: total no. of worker threads. A thread will only check each `w`th id with phase `i`.
        :param n: the last item observed to actually exist on HackerNews.
        :param output: the queue to put the results into, or `None` when the thread exits.

    '''
    end = datetime.now() - timedelta(days=7)
    session = requests.Session()
    last_debug = n - i
    for id in range(n - i, 0, -w):
        try:
            res = session.get(HN_API_URL.format(id)).json()
            if res is None or res.get('type') != 'story' or res.get('deleted'):
                # HackerNews has a single id namespace for articles and comments;
                # in this loop, we're only interested in the former.
                continue
            time = datetime.fromtimestamp(res['time'])
            if last_debug - id > 1000:
                print('worker #{} - item #{} - {}'.format(i, id, time))
                last_debug = id
            if time < end:
                break
            output.put({
                'id': res['id'],
                'score': res['score'],
                'url': res.get('url') or None,
                'time': str(time),
                'text': res.get('text', '') if not res.get('url') else get_text(session, res['url']),
                'comments': list(traverse_comments(session, res)),
            })
        except Exception:
            pass
    print('worker #{} - done'.format(i))
    output.put(None)

In [5]:
w = 10
n = requests.get(HN_API_MAX).json()
print('Starting from {}'.format(n))

data = dict()
output = multiprocessing.Queue()
processes = [multiprocessing.Process(target=get_articles, args=(i, w, n, output)) for i in range(w)]
for p in processes:
    p.start()

try:
    while w:
        res = output.get()
        if res is None:
            w -= 1
        else:
            data[res.pop('id')] = res
    for p in processes:
        p.join()
finally:
    with open('hn-corpus.txt', 'w') as file:
        json.dump(data, file)

Starting from 15444717
worker #7 - item #15443690 - 2017-10-10 20:34:25
worker #3 - item #15443704 - 2017-10-10 20:35:44
worker #9 - item #15443678 - 2017-10-10 20:33:32
worker #6 - item #15443691 - 2017-10-10 20:34:26
worker #5 - item #15443612 - 2017-10-10 20:25:50
worker #1 - item #15443656 - 2017-10-10 20:30:37
worker #0 - item #15443607 - 2017-10-10 20:25:26
worker #8 - item #15443629 - 2017-10-10 20:28:23
worker #2 - item #15443675 - 2017-10-10 20:33:08
worker #4 - item #15443633 - 2017-10-10 20:28:38
worker #7 - item #15442600 - 2017-10-10 18:28:24
worker #3 - item #15442614 - 2017-10-10 18:30:16
worker #5 - item #15442572 - 2017-10-10 18:24:30
worker #6 - item #15442601 - 2017-10-10 18:28:34
worker #8 - item #15442599 - 2017-10-10 18:28:19
worker #2 - item #15442665 - 2017-10-10 18:36:00
worker #0 - item #15442557 - 2017-10-10 18:22:31
worker #4 - item #15442623 - 2017-10-10 18:31:02
worker #7 - item #15441590 - 2017-10-10 16:33:23
worker #1 - item #15442636 - 2017-10-10 18:32:

In [6]:
with open('hn-corpus.txt', 'r') as file:
    data = json.load(file)
len(data)

6890

In [8]:
min(v['time'] for v in data.values())

'2017-10-03 22:39:13'

Прежде чем что-то делать, из статей и комментариев следует выкинуть HTML-разметку и капитализацию.

In [10]:
tags = re.compile(r'(<[^>]*>|\s)+')
for k, v in data.items():
    v['text'] = html.unescape(tags.sub(' ', v['text'].lower()))
    v['comments'] = list(map(lambda x: html.unescape(tags.sub(' ', x.lower())), v['comments']))

Бренд выберем, например, Google. Здесь есть проблема: как понять, что статья относится именно к нему, а не, скажем, мельком упоминает его в контексте новости о каком-нибудь конкуренте? Задача выглядит довольно сложной, так что решать ее не будем, а просто выберем статьи с хоть одним упоминанием.

In [11]:
brand_data = {k: v for k, v in data.items() if re.search(r'\bgoogle\b', v['text'])}

### 2. Исследование выборки

**Сколько постов в собранном корпусе?**

In [12]:
len(brand_data)

222

**Из каких источников состоит корпус?**

In [14]:
def hostname(url):
    host = urlparse(url).hostname
    if host.startswith('www.'):
        return host[4:]
    return host

hosts = collections.Counter(hostname(v['url']) for k, v in brand_data.items() if v['url'])
for host, count in hosts.most_common():
    print('{: >30} {}'.format(host, count))

                  theverge.com 16
                techcrunch.com 8
                   blog.google 8
             chrome.google.com 7
               theguardian.com 6
               arstechnica.com 6
               play.google.com 6
               docs.google.com 5
                  engadget.com 5
              store.google.com 5
          translate.google.com 5
            washingtonpost.com 4
                    recode.net 4
                 bloomberg.com 4
                    github.com 4
                    medium.com 3
               venturebeat.com 3
                   guardian.ng 3
                      cnbc.com 3
                hackernoon.com 3
                  mashable.com 3
                    google.com 3
                     wired.com 3
                   reuters.com 2
               plus.google.com 2
                   gizmodo.com 2
             groups.google.com 2
            newsroom.intel.com 2
              drive.google.com 2
              sites.google.com 2
         

**Какие слова (не считая стоп-слов) встречаются чаще всего?**

Небольшое отступление: у nltk очень странное поведение по отношению к стоп-словам. Когда строили их список, апостроф явно считали разделителем. Например, там есть такие "огрызки", как `wasn` (очевидно, от `wasn't`):

In [15]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords[-10:]

['ma',
 'mightn',
 'mustn',
 'needn',
 'shan',
 'shouldn',
 'wasn',
 'weren',
 'won',
 'wouldn']

`nltk.word_tokenize` же осведомлен о правиле `<P> not -> <P>n't` и выделяет `n't` отдельным токеном:

In [16]:
nltk.word_tokenize("mustn't")

['must', "n't"]

Поэтому при токенизации апостроф следует заменить на пробел.

In [17]:
texts = []
for v in brand_data.values():
    texts.append(v['text'])
    texts.extend(v['comments'])

words = collections.Counter(w for t in texts for w in nltk.word_tokenize(t.replace("'", " ")) if w.isalpha() and w not in stopwords)
for w, n in words.most_common(20):
    print('{: >20} {}'.format(w, float(n) / len(words)))

              google 0.12016622417176498
                like 0.05436915618146139
               https 0.048482050098118436
               would 0.04778944938243103
               phone 0.04755858247720189
               could 0.041786909846473506
                 use 0.039824541152025855
                 one 0.039824541152025855
                 get 0.0335911347108392
                even 0.03312940090038093
              people 0.030474431490245875
                also 0.02908923005887106
               think 0.028512062795798224
             android 0.02816576243795452
               pixel 0.02724229481703798
                time 0.025857093385663166
              really 0.025279926122590328
          headphones 0.02504905921736119
               still 0.024356458501673783
               fetch 0.023432990880757242


**Из всех собранных ссылок, сколько ведут на блоги и новостные издания, а сколько – на github и другие неновостные издания?**

Сразу же возникает вопрос &mdash; как отличать "новостные издания" от "неновостных изданий"? Для некоторых доменов, таких как `medium.com`, корректный класс известен, но в общем случае сайты отличить довольно сложно. Некоторые люди к тому же, например, ведут блоги на GitHub, используя GitHub Pages или даже просто Markdown файлы &mdash; кроме как вручную это не классифицировать.

In [18]:
KNOWN_NEWS = [
    'businessinsider.com', 'forbes.com', 'wired.com', 'bloomberg.com',
    'techcrunch.com', 'theverge.com', 'arstechnica.com', 'theguardian.com',
    'engadget.com', 'washingtonpost.com', 'venturebeat.com', 'reuters.com',
]

KNOWN_BLOGS = [
    'medium.com', 'hackernoon.com', '.blogspot.com',
]

def host_category(host):
    if 'news.' in host:
        return 'news'
    if 'blog.' in host:
        return 'blog'
    for k in KNOWN_NEWS:
        if host == k or (k.startswith('.') and host.endswith(k)):
            return 'news'
    for k in KNOWN_BLOGS:
        if host == k or (k.startswith('.') and host.endswith(k)):
            return 'blog'

collections.Counter(host_category(h) for h in hosts.elements())

Counter({'news': 62, 'blog': 18, None: 122})